# **1. Cel ćwiczenia**

Nieodłącznym elementem każdego systemu radarowego krótkiego zasięgu (Short Range Radar) projektowanego dla przemysłu automotive jest algorytm śledzący. Ma on za zadanie zwrócenie wysokopoziomowej informacji o poruszających się obiektach w otoczeniu czujnika, w postacji trójwymiarowych Bounding Boxów, na podstawie detekcji radarowych.

Celem tego ćwiczenia jest zaprojektowanie oraz weryfikacja prostego algorytmu śledzącego opartego na danych z wirtualnego czujnika radarowego wygenerowanych z symulatora CARLA. 


# **2. Wymagania wstępne**

Do wykonania opisanych ćwiczeń przydatna będzie podstawowa wiedza na temat właściwości czujników radarowych wykorzystywanych w przemyśle samochodowym oraz na temat macierzy transformacji, które będą wykorzystywane do przekształceń pomiędzy różnymi układami referencyjnymi. 



# **3. Opis stanowiska laboratoryjnego**

Instrukcja do laboratorium jest napisana w formie notebooka jupyterowego, w którym w kilku miejscach będzie wymagane napisanie krótkich fragmentów kodu. Każde jest odpowiednio oznaczone tekstem - **PUT YOUR CODE HERE**. Notebook można uruchomić lokalnie instalując paczkę jupyterlab lub korzystając z przeglądarkowej aplikacji Google Colab. 

Na potrzeby tego laboratorium w środowisku wirtualnym CARLA przeprowadzono testową symulację, z której wygenerowano 146 ramek danych zawierających: dane z wirtualnych czujników, dane dotyczące pojazdów znajdujących się w testowej symulacji oraz informacje na temat Bounding Boxów wygenerowanych dla tych pojazdów. 

## **3.1. CARLA**

![carla](https://drive.google.com/uc?id=1QIa0j6oOeYnTyUrj872EmQpPn2EsSLEi)

Jest to darmowy symulator stworzony na potrzeby rozwoju i walidacji algorytmów systemów aktywnego bezpieczeństwa i autonomicznej jazdy. Posiada on bardzo rozbudowane środowisko wirtualne zawierające realistycznie odwzorowane modele obiektów. Dodatkowo zaimplementowane są w nim konfigurowalne modele symulacyjne czujników stosowanych w przemyśle automotive, w szczególności:
* model kamery
* model lidaru
* model radaru

Dzięki wirtualnym sensorom oraz innym przydatnym funkcjonalościom CARLI (np. sterowanie pogodą czy ilością pojazdów w symulacji) możliwe jest przeprowadzenie wirtualnej walidacji algorytmów, czyli ich testowanie i weryfikacja przeprowadzane w całości w środowisku wirtualnym. Wirtualna walidacja staje się coraz bardziej wykorzystywa w przemyśle automotive, gdyż w środowisku wirtualnym jest możliwe wygenerowanie dowolnego scenariusza drogowego. Dzięki temu rzeczywiste rozwiązanie może być przetestowane w bardzo trudnych sytuacjach często niemożliwych (lub bardzo trudnych i kosztownych) do przeprowadzenia na drodze.

## **3.3. Układy współrzędnych**

Na potrzeby tego laboratorium definiuje się następujące układy referencyjne:
* Globalny układ współrzędnych (**GCS**). Jest to układ współrzędnych, którego początkiem jest punkt $[0, 0, 0]$ symulatora. Wyrażone są w nim pozycje, orientacje oraz prędkości pojazdów.
* Układ współrzędnych sensora (**SCS**) - jest to układ współrzędnych zdefiniowany w punkcie, w którym zamocowany jest czujnik. Wyróżniamy dwa układy SCS:
    * SCS dla czujnika radarowego (detekcje)
    * SCS dla kamery (piksele)

## **3.4. Dane**

Dane dla każdej ramki są zapisane w formie słownika pythonowego. Pola słownika zawierają następujące informacje:
* Dane z czujników radarowych, zapisane w 
    * macierz transformacji z radarowego układu **SCS** do układu **GCS**
    * detekcje radarowe dla obu modeli (wbudowanego i wyidealizowanego) zapisane w postaci macierzy typu **numpy.array**, gdzie każdy wiersz to osobna detekcja a każda kolumna to kolejno:
        * odległość radialna $r [m]$ 
        * prędkość radialna detekcji (prędkość względna) $v  [\frac{m}{s}]$
        * kąt azymutu $\phi [rad]$
        * kąt elewacji $\theta [rad]$
       
* Dane z kamery:
    * macierz transformacji z układu **SCS** kamery do układu **GCS**
    * piksele obrazu dla danej ramki
* Dane na temat pojazdu hosta:
    * ID 
    * pozycja $p_h = [p_{hx}, p_{hy}, p_{hz}] [m]$ 
    * prędkość $v_h = [v_{hx}, v_{hy}, v_{hz}] [\frac{m}{s}]$
    * orientacja $\alpha_h = [roll_{hx}, pitch_{hy}, yaw_{hz}] [rad]$
* Lista pythonowa z danymi na temat wszystkich pojazdów (31 pojazdów wliczając pojazd hosta) znajdujących się w wirtualnym scenariuszu:
    * ID$^m$ 
    * pozycja $p_t^m = [p_{tx}^m, p_{ty}^m, p_{tz}^m]$ 
    * prędkość $v_t^m= [v_{tx}^m, v_{ty}^m, v_{tz}^m] [\frac{m}{s}]$
    * orientacja $\alpha_t^m = [roll_{tx}^m, pitch_{ty}^m, yaw_{tz}^m] [rad]$, gdzie $m$ to indeks wskazujący na dany pojazd w liście
* Lista pythonowa z informacjami na temat Bounding Boxów:
    * ID pojazdu, do którego dany Boundng Box jest przypisany
    * aktualne współrzędne Bounding Boxa wyrażone w układzie współrzędnych kamery

# **4. Przebieg ćwiczenia**

## **4.1. Instalacja pakietów**

Do przeprowadzenia tego ćwiczenia będą wymagane następujące moduły języka Python:
* pickle oraz Path - do wczytania zapisanych danych do słownika Pythonowego
* numpy - do operacji macierzowych (transformacje pomiędzy układami współrzędnych)
* pygame - do wyświetlania rezultatów (obrazu z kamery, detekcji radarowych, wykrytych Bounding Boxów)

In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pathlib2

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


## **4.2. Inicjalizacja skryptu**

In [4]:
import pickle
from pathlib2 import Path
import pygame
import numpy as np

VIEW_WIDTH = 1920 // 2
VIEW_HEIGHT = 1080 // 2
VIEW_FOV = 90

BB_COLOR = (248, 64, 24)
DET_COLOR = (255, 255, 0)

pygame.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


(6, 0)

## **4.3. Wczytanie danych**

Teraz należy wczytać pojedynczą ramkę danych. Można także wyświetlić pola składowe słownika Pythonowego:
* w przypadku pakietu jupyterlab uruchamianego lokalnie należy umieścić folder **data** w tej samej lokalizacji co notebook
* w przypadku korzystania z Google Colab należy:
  * w domyślej lokalizacji plików utworzyć folder **data** (opcja New Folder)
  * klikając na nowo utworzony folder wybrać opcję **Upload** i załadować z lokalnego dysku wszystkie ramki danych. 

In [7]:
frame = 101
file = Path('data/') / ('frame' + str(frame * 10) + '.dat')
if not file.parent.exists():
    file.parent.mkdir(parents=True)

with open(str(file), 'rb') as in_file:
    logging_data = pickle.load(in_file)
in_file.close()

print(logging_data.keys())
print(logging_data['radar'].keys())
print(logging_data['radar']['raw_data'].keys())
print(logging_data['camera'].keys())
print(logging_data['labels'].keys())
print(logging_data['host'].keys())
print(len(logging_data['actors']))
print(logging_data['actors'][0].keys())

dict_keys(['radar', 'camera', 'labels', 'host', 'actors'])
dict_keys(['raw_data', 'trans_matrix_radar_to_world'])
dict_keys(['carla_model', 'ideal_model'])
dict_keys(['raw_data', 'trans_matrix_camera_to_world', 'calibration'])
dict_keys(['bounding_box', 'id'])
dict_keys(['id', 'position', 'rotation', 'velocity'])
31
dict_keys(['id', 'position', 'rotation', 'velocity'])


## **4.4. Widok z kamery**

W każdej zapisanej ramce dostępny jest także obraz z kamery, na który w późniejszych etapach zostaną nałożone detekcje radarowe oraz wykryte przez algorytm śledzący Bounding Boxy. 

In [8]:
image = logging_data['camera']['raw_data']
surface = pygame.surfarray.make_surface(image.swapaxes(0, 1))
display = pygame.display.set_mode((VIEW_WIDTH, VIEW_HEIGHT), pygame.HWSURFACE | pygame.DOUBLEBUF)
display.blit(surface, (0, 0))
pygame.display.flip()
pygame.event.pump()

## **4.5. Detekcje radarowe**

W zapisanych danych dostępne są dwa zestawy detekcji: 
* z modelu CARLI, którego detekcje zostaną wykorzystane w algorytmie śledzenia
* z modelu idealnego służącego jako narzędzie do weryfikacji poprawności przeprowadzonych transformacji układów współrzędnych

W obu modelach symulacyjnych dostępny jest także pomiar elewacji. Aktualnie w rzeczywistych czujnikach radarowych pomiar elewacji jest albo niedostępny, albo wyznaczony z bardzo dużym błędem. Warto także dodać, że detekcje radarowe są wyznaczone w konwencji prawoskrętnego układu współrzędnych. 

In [9]:
carla_detections = logging_data['radar']['raw_data']['carla_model']
print(carla_detections.shape)
distance = carla_detections[:, 0]
velocity = carla_detections[:, 1]
azimuth = carla_detections[:, 2]
elevation = carla_detections[:, 3]

ideal_detections = logging_data['radar']['raw_data']['ideal_model']
print(ideal_detections.shape)
distance = ideal_detections[:, 0]
velocity = ideal_detections[:, 1]
azimuth = ideal_detections[:, 2]
elevation = ideal_detections[:, 3]

(43, 4)
(103, 4)


## **4.6. Zadanie nr. 1**

Na obrazie z kamery można wyświetlać (po odpowiednich transformacjach) detekcje radarowe. Wyświetlenie detekcji znacznie ułatwi przetestowanie algorytmu śledzącego, gdyż będzie można zobaczyć które obiekty mogły zostać potencjalnie wykryte. 

Dokonaj transformacji detekcji radarowych do układu **CSC**. Uzyskane rezultaty wyświetl przy użyciu modułu 
pygame.

![model](https://drive.google.com/uc?id=1Mie2rFG-QZNx3P_roA6ipyOrkn6HUyDl)

### **Sprawozdanie cz. 1**

Wykorzystaj najpierw model idealny do sprawdzenia poprawności transformacji układów współrzędnych. Następnie powtórz tą samą operację dla modelu CARLI. Uzyskane rezultaty wyświetl na obrazie z kamery przy użyciu modułu pygame.

### Przekształcenie detekcji do układu układu kartezjańskiego

Zanim będzie można skorzystać z macierzy transformacji należy przekształcić detekcje radarowe wyrażone we współrzędnych sferycznych do układu kartezjańskiego. 

Ważna informacją jest tutaj to, że macierze transformacji zostały wygenerowane dla lewoskrętnego układu współrzędnych, gdyż taki właśnie układ jest wykorzystywany w CARLI. Przekształcenie jest proste, gdyż w przypadku współrzędnych (x, y, z) należy tylko zamienić kierunek osi Y. 

Dodatkowo konieczne jest dodanie czwartego wymiaru do wektora XYZ. Jest to tzw. przekształcenie do współrzędnych jednorodnych. W przypadku przekształceń pomiędzy układami współrzędnych z wykorzystaniem macierzy transformacji czwarta współrzędna ma zawsze wartość 1.

In [10]:
# detections = carla_detections
detections = ideal_detections
print(detections.shape)
distance = detections[:, 0]
azimuth = detections[:, 2]
elevation = detections[:, 3]

detections_xyz = np.zeros((detections.shape[0], detections.shape[1]))
detections_xyz[:, 0] = distance * np.cos(azimuth) * np.cos(elevation)
detections_xyz[:, 1] = -distance * np.sin(azimuth) * np.cos(elevation)
detections_xyz[:, 2] = distance * np.sin(elevation)
detections_xyz[:, 3] = 1

(103, 4)


### Przygotowanie macierzy transformacji

Do przekształcenia detekcji radarowych zostaną wykorzystane dwie macierze transformacji.

In [11]:
trans_matrix_radar_to_world = logging_data['radar']['trans_matrix_radar_to_world']
trans_matrix_world_to_camera = np.linalg.inv(logging_data['camera']['trans_matrix_camera_to_world'])

### Wyznaczenie złożonej macierzy transformacji

Korzystając ze zdefiniowanych wyżej macierzy transformacji należy wyznaczyć jedną macierz reprezentującą złożone przekształcenie. Należy tutaj pamiętać, że transformacje łączy się od prawej do lewej. 

Wskazówka - wykorzystaj operacje mnożenia macierzowego **dot** dostępną w pakiecie numpy

In [0]:
# PUT YOUR CODE HERE
# trans_matrix_radar_to_camera = ...

### Przekształcenie punktów z układu współrzędnych radaru do układu współrzędnych kamery

Korzystając z wyznaczonej złożonej macierzy transformacji należy przekształcić wektor punktów **detections_xyz** do układu współrzędnych kamery.

In [0]:
# PUT YOUR CODE HERE
# detections_camera_xyz = ...

### Zrzutowanie punktów XYZ na płaszczyznę kamery

Aby możliwe było wyświetlenie współrzędnych XYZ na obrazie kamery należy te współrzędne odpowiednio zrzutować. Na tym etapie pozbywamy się już czwartego wymiaru danych. Był on dodany tylko na potrzeby transformacji.

In [0]:
detections_camera_y_minus_z_x = np.concatenate([detections_camera_xyz[1, :], -detections_camera_xyz[2, :],
                                                            detections_camera_xyz[0, :]])
detections_camera = np.transpose(np.dot(logging_data['camera']['calibration'], detections_camera_y_minus_z_x))
detections_camera = np.concatenate([detections_camera[:, 0] / detections_camera[:, 2],
                                    detections_camera[:, 1] / detections_camera[:, 2], detections_camera[:, 2]],
                                    axis=1)


### Wyświetlenie przekształconych detekcji na obrazie kamery

In [0]:
detection_surface = pygame.Surface((VIEW_WIDTH, VIEW_HEIGHT))
detection_surface.set_colorkey((0, 0, 0))
for det in detections_camera:
  det_tuple = (int(det[0, 0]), int(det[0, 1]))
  pygame.draw.circle(detection_surface, DET_COLOR, det_tuple, 2, 0)

display.blit(detection_surface, (0, 0))
pygame.display.flip()
pygame.event.pump()

## **4.7. Bounding Boxy**

W zapisanych danych dostępne są Bounding Boxy wygenerowane dla każdego z pojazdów (łącznie z pojazdem hosta).

### Wyświetlenie Bounding Boxów

Jak zostało wspomniane na począ†ku współrzędne Bounding Boxów są już przekształcone do układu współrzędnych kamery (łącznie z rzutowaniem na płaszczyznę kamery), dlatego nie są potrzebne już żadne transformacje.

Na tym etapie można wczytywać różne ramki danych (ręcznie lub w pętli), żeby zobaczyć cały przejazd hosta z przeprowadzonej symulacji. Jak można zauważyć wiele spośród wyświetlonych pojazdów (Bounding Boxów) jest nie widoczna przez radar i dlatego nie powinny być dla nich być wyświetlone labelki. 

In [0]:
bb_surface = pygame.Surface((VIEW_WIDTH, VIEW_HEIGHT))
bb_surface.set_colorkey((0, 0, 0))
bounding_boxes = logging_data['labels']['bounding_box']
for bbox in bounding_boxes:
    points = [(int(bbox[i, 0]), int(bbox[i, 1])) for i in range(8)]
    # draw lines
    # base
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[1])
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[1])
    pygame.draw.line(bb_surface, BB_COLOR, points[1], points[2])
    pygame.draw.line(bb_surface, BB_COLOR, points[2], points[3])
    pygame.draw.line(bb_surface, BB_COLOR, points[3], points[0])
    # top
    pygame.draw.line(bb_surface, BB_COLOR, points[4], points[5])
    pygame.draw.line(bb_surface, BB_COLOR, points[5], points[6])
    pygame.draw.line(bb_surface, BB_COLOR, points[6], points[7])
    pygame.draw.line(bb_surface, BB_COLOR, points[7], points[4])
    # base-top
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[4])
    pygame.draw.line(bb_surface, BB_COLOR, points[1], points[5])
    pygame.draw.line(bb_surface, BB_COLOR, points[2], points[6])
    pygame.draw.line(bb_surface, BB_COLOR, points[3], points[7])
display.blit(bb_surface, (0, 0))
pygame.display.flip()
pygame.event.pump()

## **4.8. Rzeczywisty algorytm śledzący**

Rzeczywity tracker opiera swoje wyniki tylko na detekcjach radarowych. Estymuje on wielkość i pozycję Bounding Boxa dla **poruszających się** obiektów znajdujących się w polu widzenia radaru. Często wykorzystuje do tego celu różne odmiany filtru Kalmana. 

W ramach wirtualnej walidacji często na początku projektu przygotowuje się uproszczone modele symulacyjne mające naśladować rzeczywiste algorytmy. Taki uproszczony model musi zwracać dane w takim samym formacie jak rzeczywiste rozwiązanie. Może jednak wyliczyć swoje detekcje korzystając z zupełnie innych informacji. Nie musi także (przynajmniej na początku) zwracać porównywalnych wyników. Chodzi tutaj np. o to, żeby sprawdzić poprawność przepływu danych dla całego systemu aktywnego bezpieczeństwa - od percepcji do sterowania - w środowisku wirtualnym a nie o to, by idealne odwzorować detekcje trackerowe w symulatorze.


Zadaniem projektowanego w ramach tego laboratorium rozwiązania będzie przygotowanie uproszczonego algorytmu śledzącego.  

W rzeczywistości tracker nie ma informacji o Bounding Boxach pojazdów znajdujących się w jego otoczeniu. W przypadku tego ćwiczenia do projektu trackera mogą zostać wykorzystane wszystkie dostępne w logach dane.  



## **4.9. Zadanie nr. 2**

Przygotuj uproszczony model symulacyjny naśladujący radarowy algorytm śledzący. 

**Założenia**:
* Model powinien korzystać z detekcji radarowych wygenerowanych przez model CARLI
* Model powinien zwrócić listę Bounding Boxów dla **poruszających się** obiektów (o niezerowej prędkości względnej) wykrytych przez czujnik radarowy
* Wykryte Bounding Boxy należy wyświetlić na obrazie kamery. Pozostałe nie powinny być wyświetlone. 


**Wskazówki**:
* Model radaru powinien dla poruszającego się pojazdu zwrócić kilka detekcji o podobnych do siebie parametrach (prędkości względnej, odległości, kącie). Prędkość względna tych detekcji będzie jednak wyraźnie większa (lub mniejsza) od detekcji pochodzących od obiektów statycznych. Możliwe jest więc pogrupowanie detekcji (przynajmniej w przybliżony sposób) i przypisanie ich do danego obiektu, np. poprzez porównanie średniej odległości dostępnej w detekcji z odległością danego obiektu do samochodu hosta. 
* Obiekty statyczne także będą mieć niezerową prędkość względną w przypadku, gdy pojazd hosta się porusza.
* Zarówno obiekt jak i jego Bounding Box są powiązane ze sobą na podstawie ID, które jest zapisane w obu zbiorach danych. 




In [0]:
visible_bounding_boxes = bounding_box_data['bounding_box']

# PUT YOUR CODE HERE

## **4.10. Wyświetlenie rezultatów na obrazie kamery**

Wykryte Bounding Boxy należy wyświetlić na obrazie z kamery. 


In [0]:
bb_surface = pygame.Surface((VIEW_WIDTH, VIEW_HEIGHT))
bb_surface.set_colorkey((0, 0, 0))
for bbox in visible_bounding_boxes:
    points = [(int(bbox[i, 0]), int(bbox[i, 1])) for i in range(8)]
    # draw lines
    # base
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[1])
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[1])
    pygame.draw.line(bb_surface, BB_COLOR, points[1], points[2])
    pygame.draw.line(bb_surface, BB_COLOR, points[2], points[3])
    pygame.draw.line(bb_surface, BB_COLOR, points[3], points[0])
    # top
    pygame.draw.line(bb_surface, BB_COLOR, points[4], points[5])
    pygame.draw.line(bb_surface, BB_COLOR, points[5], points[6])
    pygame.draw.line(bb_surface, BB_COLOR, points[6], points[7])
    pygame.draw.line(bb_surface, BB_COLOR, points[7], points[4])
    # base-top
    pygame.draw.line(bb_surface, BB_COLOR, points[0], points[4])
    pygame.draw.line(bb_surface, BB_COLOR, points[1], points[5])
    pygame.draw.line(bb_surface, BB_COLOR, points[2], points[6])
    pygame.draw.line(bb_surface, BB_COLOR, points[3], points[7])
display.blit(bb_surface, (0, 0))
pygame.display.flip()
pygame.event.pump()

## **Destrukcja modułu pygame**

Po wykonaniu tej komendy zostanie zamknięte okienko modułu pygame i konieczne będzie ponowne przeprowadzenie jego inicjalizacji.

In [0]:
pygame.quit()

## **4.11. Sprawozdanie cz. 2**

Przetestuj zaprojektowany algorytm dla wszystkich ramek oraz pod kątem różnych scenariuszy. Przygotuj raport końcowy, w którym zamieszczone będą następujące informacje: 
* ogólny opis i cel zadania
* założenia wstępne postawione przed projektowanym algorytmem
* schemat blokowy i opis zaimplementowanego rozwiązania:
* wyniki oraz wnioski zebrane z testów
* zalety i wady przygotowanego algorytmu
* potencjalne usprawnienia, które mogłyby być dodane do przyszłych wersji, które uczyniłyby model symulacyjny bardziej rzeczywistym